In [1]:

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models.ldamodel import LdaModel as Lda
from gensim import corpora
import string
import os
import codecs
import pandas as pd

# Function to remove stop words from sentences, punctuation & lemmatize words. 
def clean(doc):
    exclude = set(string.punctuation)
    translate_table = dict((ord(char), None) for char in string.punctuation)
    no_punct = doc.lower().translate(translate_table)
    stop_free = " ".join([i for i in no_punct.split() if i not in stop])
    normalized = " ".join(lemma.lemmatize(word,'v') for word in stop_free.split())
    
    x = normalized.split()
    y = [s for s in x if len(s) > 2]
    return y


In [10]:
rawPost=pd.read_csv("TPostRaw.csv", names = ['text'], sep = "\t")

In [11]:
rawPost



,text
0,Huge brain fog - help me remember a website I ...
1,SaaS/web based business people: what's your te...
2,Help me understand this client.. I have a clie...
3,Any Work/Lifestyle Alternatives To YT? Could a...
4,Is my idea of a courier service in my tiny tow...
5,What to do with 13k in a tourist city ? Hi! I ...
6,How to find bloggers? I'm wondering how you gu...
7,Low Value Commercial Real Estate Loan I have a...
8,To those who have sent/received thank you card...
9,Is it a good idea to use Alibaba to find suppl...


In [12]:
# Cleaning 
stop = set(stopwords.words('english'))
lemma = WordNetLemmatizer()

#There are \n\n in data above, need to delete them
rawPost['text'] = rawPost['text'].replace(r'\\n\\n|\\n',' ', regex=True)

# Delete URL, website etc.
rawPost['text'] = rawPost['text'].replace(r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))""",' ', regex=True)

cleanPost = [clean(doc) for doc in rawPost['text']]


In [17]:
print(cleanPost[0:100]) 



[['huge', 'brain', 'fog', 'help', 'remember', 'website', 'use', 'frequent', 'hey', 'know', 'know', 'answer', 'use', 'visit', 'website', 'people', 'pitch', 'products', 'andor', 'startups', 'make', 'unique', 'special', 'account', 'put', 'products', 'platform', 'recently', 'sign', 'product', 'hunt', 'think', 'look', 'completely', 'different', 'remember', 'product', 'hunt', 'use', 'couple', 'years', 'ago', 'undergo', 'redesign', 'another', 'website', 'think', 'ahh', 'good', 'call', 'aware', 'think', 'harder', 'yeah', 'look', 'like', 'product', 'hunt', 'site', 'mind', 'thank', 'view', 'old', 'versions', 'product', 'hunt', 'heres', 'one', 'earliest', 'ones', 'find', 'hacker', 'news', 'purchase', 'combinator'], ['saasweb', 'base', 'business', 'people', 'whats', 'technique', 'calculate', 'cost', 'per', 'customer', 'acquisition', 'noobie', 'google', 'analytics', 'also', 'use', 'stripe', 'payment', 'process', 'mailchimp', 'mail', 'ads', 'super', 'limit', 'reddit', 'google', 'anyone', 'experience

In [14]:
# Find the most frequent words and exclude NEUtral them. My bias!!! May be work more on that?
import itertools
flattened_cleanPost = list(itertools.chain(*cleanPost))

from collections import Counter
word_counts = Counter(flattened_cleanPost)
top_three = word_counts.most_common(200)
print(top_three)

[('get', 355306), ('make', 265322), ('would', 258959), ('like', 252912), ('business', 244373), ('work', 229912), ('people', 218258), ('think', 193464), ('dont', 179903), ('want', 178593), ('use', 176140), ('time', 167837), ('need', 163974), ('start', 162334), ('know', 160239), ('one', 158595), ('good', 152501), ('look', 150288), ('thank', 126554), ('find', 123736), ('take', 123581), ('market', 123199), ('sell', 123124), ('company', 122496), ('also', 118726), ('see', 115862), ('really', 115230), ('product', 112430), ('could', 111505), ('say', 110686), ('idea', 109193), ('much', 108317), ('something', 106859), ('try', 106370), ('money', 105799), ('help', 104873), ('lot', 100087), ('youre', 99749), ('pay', 95505), ('way', 94985), ('ive', 91592), ('give', 90515), ('well', 89979), ('post', 85603), ('great', 81134), ('site', 81083), ('first', 80625), ('even', 80508), ('build', 76757), ('come', 74587), ('website', 73983), ('service', 72859), ('thats', 72499), ('things', 72300), ('buy', 71985)

In [15]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(cleanPost)


In [18]:
#After printing the most frequent words of the dictionary, I found that few words which are mostly content neutral words are also present in the dictionary. 
# These words may lead to modeling of “word distribution”(topic) which is neutral and do not capture any theme or content. 
# I made a list of such words and filtered all such words.
stoplist = set('get would like want hey might may without also make want put etc actually else far definitely youll\' didnt\' isnt\' theres since able maybe without may suggestedsort never isredditmediadomain userreports far appreciate next think know need look please one null take dont\' want\' could able ask well best someone sure lot thank also anyone really something give years use make all ago people know many call include part find become '.split())
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
dictionary.filter_tokens(stop_ids)


In [ ]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in cleanPost]

In [18]:
#Creating the object for LDA model using gensim library & Training LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=2000, id2word = dictionary, passes=50, iterations=500)
#ldafile = open('lda_model_sym_wiki.pkl','wb')
#cPickle.dump(ldamodel,ldafile)
#ldafile.close()

KeyboardInterrupt: 

In [ ]:
#Print all the 50 topics
for topic in ldamodel.print_topics(num_topics=50, num_words=10):
print topic[0]+1, " ", topic[1],"\n"